In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests

### Mailchimp API?

In [ ]:
mailchimp_api_key = 

In [ ]:
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError

try:
  client = MailchimpMarketing.Client()
  client.set_config({
    "api_key": mailchimp_api_key,
    "server": "us4"
  })

  response = client.lists.get_all_lists()
#   print(response)
except ApiClientError as error:
  print("Error: {}".format(error.text))

In [ ]:
import hashlib

In [ ]:
# response['lists']

In [ ]:
# mailchimp_list = 

In [ ]:
# members = client.lists.get_list_members_info(mailchimp_list, count=1)

In [ ]:
# members

In [ ]:

# subscriber = 

## existing functions...

In [ ]:
import os
mailchimp_api_key = 
eric_g_maps_key = 
mailchimp_list = 

In [ ]:
boundaries = gpd.read_file('./Political Boundaries v1/LA Political Boundaries.shp')
cogs = gpd.read_file('./Regional_Council_Districts_–_SCAG_Region.geojson')
cogs = cogs[['SUBREGION', 'geometry']].rename(columns={'SUBREGION':'COG'})
senators = pd.read_csv('./senators.csv').set_index('District').fillna('')
assemblymembers = pd.read_csv('./assemblyJune21.csv')

try:
    client = MailchimpMarketing.Client()
    client.set_config({
        "api_key": mailchimp_api_key,
        "server": "us4"
      })

    response = client.lists.get_all_lists()
#   print(response)
except ApiClientError as error:
    logging.info("Error: {}".format(error.text))

sfa_list_id = response['lists'][0]['id']

merge_fields = {'NC': 'MMERGE17', 'LA City': 'MMERGE11', 'LA County': 'MMERGE12', 'State Asse': 'MMERGE13',
               'State Sena': 'MMERGE14', 'Fed House': 'MMERGE15', 'COG':'MMERGE16'}

def geocode (address, key=eric_g_maps_key):
    # print(f'key:{key}')
    '''Geocode using Google Maps API (Eric's key)'''
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={key}'
    print(url)
    r = requests.get(url)
    return r.json()

In [ ]:
import logging
logging.basicConfig(filename='geocoder.log', encoding='utf=8', level=logging.INFO)

In [ ]:
def sanitize_address(address):
    address = address.replace('#', 'Unit ')
    return address

In [ ]:
def geocode_subscriber(subscriber):

    # return subscriber
    address_fields = subscriber['merge_fields']['ADDRESSYU']
    if not address_fields:
        logging.info(f'no address for: {subscriber["full_name"]}, {subscriber["email_address"]}')
        return gpd.GeoDataFrame()
        ## quick fix for missing state info
#     print(address_fields)
    if not address_fields['state'] and (address_fields['zip'][0] == '9'):
        address_fields['state'] = 'CA'
        logging.info('updating-->', end='')
    address = ' '.join(
            [address_fields['addr1'], address_fields['city'], address_fields['state'], address_fields['zip']])
    logging.info(address)
    address = sanitize_address(address)
    
    print(f'{subscriber["full_name"]}, {subscriber["email_address"]}')
    geocoded = geocode(address)
    
    if geocoded['status'] in ('INVALID_REQUEST', 'ZERO_RESULTS'):
        logging.info(f'no match for: {subscriber["full_name"]}, {subscriber["email_address"]}')
        return gpd.GeoDataFrame()
    else:
        #create Geopandas GeoDataFrame for spatial join
        gdf = gpd.GeoDataFrame([geocoded['results'][0]['geometry']['location']])
        gdf.geometry = gpd.points_from_xy(gdf['lng'], gdf['lat'])
        gdf = gdf.set_crs('EPSG:4326')
        return gdf.to_crs('EPSG:3310')

In [ ]:
la_city = gpd.read_file('./2022_boundaries/LA_CC/LA City Council Districts 2021.shp').to_crs('EPSG:3310')

In [ ]:
la_city.head(1)

In [ ]:
la_county = gpd.read_file('./2022_boundaries/LA_County_Supe/LA_Co_Supervisorial_Districts_(2021).shp').to_crs('EPSG:3310')

In [ ]:
la_county.head(1)

In [ ]:
legacy_boundaries = gpd.read_file('./Political Boundaries v1/LA Political Boundaries.shp').to_crs('EPSG:3310')

In [ ]:
legacy_boundaries.head(1)

In [ ]:
cogs = gpd.read_file('./Regional_Council_Districts_–_SCAG_Region.geojson').to_crs('EPSG:3310')

In [ ]:
cogs.head(1)

In [ ]:
la_nc = gpd.read_file('./2022_boundaries/LA_NC/City of LA Neighborhood Councils.shp').to_crs('EPSG:3310')

In [ ]:
la_nc.head(1)

In [ ]:
assemblymembers = pd.read_csv('./assemblyJune21.csv')
assemblymembers['Dist'] = assemblymembers['Dist'].apply(lambda x: x[:3]).astype('int64')
assemblymembers = assemblymembers.set_index('Dist')

In [ ]:
senators = pd.read_csv('./senators.csv').set_index('District').fillna('')

In [ ]:
def assembly_name(district):
    if district == '':
        return
    member = assemblymembers.loc[int(district), 'Member'].split(',')
    member.reverse()
    member = ' '.join(member)
    return member.strip()

In [ ]:
def senate_name(district):
    if district == '':
        return
    senator = ' '.join(senators.loc[int(district)])
    return senator.strip()

In [ ]:
def sjoin_return(sub_gdf, boundary_gdf, col, _int=True):
    boundary_gdf = boundary_gdf[[col, 'geometry']]
    joined = gpd.sjoin(sub_gdf, boundary_gdf)
    if joined.empty:
        return ''
    value = joined[col].iloc[0]
    if _int and type(value) != str:
        value = int(value)
    return str(value)

In [ ]:
merge_fields = {'NC': 'NC', 'LA City': 'LACOUNCILD', 'LA County': 'LACOUNTYD', 'State Asse': 'CAASSEMBLD',
               'State Sena': 'CASENATED', 'Fed House': 'CONDISTRIC', 'COG':'COG', 'CASENATOR': 'CASENATOR',
               'CAASSEMBLY': 'CAASSEMBLY'}

In [ ]:
def geocode_and_update_subscriber(subscriber_id):
    
    subscriber = client.lists.get_list_member(list_id=mailchimp_list,
                                              subscriber_hash=subscriber_id)

    geocoded = geocode_subscriber(subscriber)
    if not geocoded.empty:
        #add geocoded info to Mailchimp merge fields
        subscriber['merge_fields'][merge_fields['LA City']] = sjoin_return(geocoded, la_city, 'district')
        subscriber['merge_fields'][merge_fields['LA County']] = sjoin_return(geocoded, la_county, 'DISTRICT')
        subscriber['merge_fields'][merge_fields['State Asse']] = sjoin_return(geocoded, legacy_boundaries, 'State Asse')
        subscriber['merge_fields'][merge_fields['State Sena']] = sjoin_return(geocoded, legacy_boundaries, 'State Sena')
        subscriber['merge_fields'][merge_fields['Fed House']] = sjoin_return(geocoded, legacy_boundaries, 'Fed House')
        subscriber['merge_fields'][merge_fields['COG']] = sjoin_return(geocoded, cogs, 'SUBREGION')
        subscriber['merge_fields'][merge_fields['NC']] = sjoin_return(geocoded, la_nc, 'name')
        #value lookup for names
        subscriber['merge_fields'][merge_fields['CASENATOR']] = senate_name(subscriber['merge_fields'][merge_fields['State Sena']])
        subscriber['merge_fields'][merge_fields['CAASSEMBLY']] = assembly_name(subscriber['merge_fields'][merge_fields['State Asse']])


    else:
        return
    
    #update subscriber info
    try:
        # print(subscriber)
        client.lists.update_list_member(
            list_id=mailchimp_list, subscriber_hash=subscriber_id, body=subscriber)
    except ApiClientError as error:
        logging.info("Error: {}".format(error.text))
    return 

In [ ]:
# client.lists.get_list_merge_fields(mailchimp_list, count=30)

In [ ]:
def hash_email(email):
    h1 = hashlib.md5(email.encode())
    return h1.hexdigest()

In [ ]:
# manually update subscribers (backup first!)
# for offset in range(0,3800,200):
for offset in range(0,3801,200):
    logging.info(offset)
    print(offset)
    members = client.lists.get_list_members_info(mailchimp_list, count=200, offset=offset)
    for member in members['members']:
        logging.info(f'next: {member["full_name"]}, {member["email_address"]}, {member["id"]}')
        try:
            geocode_and_update_subscriber(member['id'])
        except Exception as e:
            print(e)
            logging.info(f'geocode failed for {member["full_name"]}')
            logging.info(e)